In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, RandomizedSearchCV
import xgboost as xgb
from scipy.stats import randint, uniform
import joblib
import warnings
warnings.filterwarnings('ignore')

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置中文字体
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
plt.style.use('seaborn-v0_8-whitegrid')  # 使用seaborn样式

# 创建中文字体对象
font_path = 'C:/Windows/Fonts/simhei.ttf'
if os.path.exists(font_path):
    chinese_font = fm.FontProperties(fname=font_path)
    print("使用SimHei字体文件路径")
else:
    # 回退到rcParams方法
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['axes.unicode_minus'] = False
    chinese_font = None
    print("使用rcParams设置SimHei字体")

# 创建输出目录
output_dir = 'dengue_xgboost_results'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 读取处理过的数据
print("=== 开始加载数据 ===")
df = pd.read_csv('../pre_data/processed_dengue_data.csv')

# 确保日期列为datetime类型
df['Date'] = pd.to_datetime(df['Date'])

print(f"原始数据形状: {df.shape}")
print(f"数据时间范围: {df['Date'].min()} 至 {df['Date'].max()}")
print("数据加载完成！\n")

使用SimHei字体文件路径
=== 开始加载数据 ===
原始数据形状: (1020, 8)
数据时间范围: 2016-01-01 00:00:00 至 2020-12-01 00:00:00
数据加载完成！



In [3]:
# 1. 特征工程
print("1. 特征工程")

# 确保数据按地区和日期排序
df = df.sort_values(['Region', 'Date'])

# 1.1 创建滞后特征
print("  创建滞后特征...")
for lag in range(1, 4):
    df[f'Lag{lag}_Cases'] = df.groupby('Region')['Dengue_Cases'].shift(lag)

# 1.2 创建移动平均特征
print("  创建移动平均特征...")
df['MA3_Cases'] = df.groupby('Region')['Dengue_Cases'].transform(
    lambda x: x.rolling(window=3, min_periods=1).mean())
df['MA6_Cases'] = df.groupby('Region')['Dengue_Cases'].transform(
    lambda x: x.rolling(window=6, min_periods=1).mean())
df['MA12_Cases'] = df.groupby('Region')['Dengue_Cases'].transform(
    lambda x: x.rolling(window=12, min_periods=1).mean())

# 1.3 创建季节性特征
print("  创建季节性特征...")
df['Month_Sin'] = np.sin(2 * np.pi * df['MonthNum'] / 12)
df['Month_Cos'] = np.cos(2 * np.pi * df['MonthNum'] / 12)

# 1.4 创建趋势特征
print("  创建趋势特征...")
df['Year_Num'] = df['Year'] - df['Year'].min()  # 从0开始的年份数值

# 1.5 添加增长率特征
print("  创建增长率特征...")
df['MoM_Growth'] = df.groupby('Region')['Dengue_Cases'].pct_change()
df['YoY_Growth'] = df.groupby(['Region', 'MonthNum'])['Dengue_Cases'].pct_change()

# 1.6 创建地区编码（独热编码）
print("  创建地区编码...")
region_dummies = pd.get_dummies(df['Region'], prefix='region')
df = pd.concat([df, region_dummies], axis=1)

# 1.7 处理缺失值
print("  处理缺失值...")
# 对于增长率特征，将无穷大替换为NaN然后替换为0
for col in ['MoM_Growth', 'YoY_Growth']:
    df[col] = df[col].replace([np.inf, -np.inf], np.nan)

# 填充缺失值
num_cols = df.select_dtypes(include=[np.number]).columns
df[num_cols] = df[num_cols].fillna(0)

print(f"  特征工程完成，数据形状: {df.shape}")

1. 特征工程
  创建滞后特征...
  创建移动平均特征...
  创建季节性特征...
  创建趋势特征...
  创建增长率特征...
  创建地区编码...
  处理缺失值...
  特征工程完成，数据形状: (1020, 36)


In [4]:
# 2. 准备训练、验证和测试数据
print("\n2. 准备训练、验证和测试数据")

# 2.1 定义特征和目标列
target_col = 'Dengue_Cases'

# 基础特征集
feature_cols = [
    # 时间特征
    'Year_Num', 'MonthNum', 'Month_Sin', 'Month_Cos',
    # 滞后特征
    'Lag1_Cases', 'Lag2_Cases', 'Lag3_Cases',
    # 移动平均特征
    'MA3_Cases', 'MA6_Cases', 'MA12_Cases',
    # 增长率特征
    'MoM_Growth', 'YoY_Growth'
]

# 添加地区编码特征
region_cols = [col for col in df.columns if col.startswith('region_')]
feature_cols.extend(region_cols)

print(f"  选择的特征数量: {len(feature_cols)}")
print(f"  前5个特征: {feature_cols[:5]}")

# 2.2 按时间分割数据
# 使用最后12个月作为测试集
test_start_date = df['Date'].max() - pd.DateOffset(months=11)
print(f"  测试集开始日期: {test_start_date}")

# 分割数据
train_df = df[df['Date'] < test_start_date].copy()
test_df = df[df['Date'] >= test_start_date].copy()

print(f"  训练集形状: {train_df.shape}")
print(f"  测试集形状: {test_df.shape}")

# 2.3 准备X和y
X_train = train_df[feature_cols]
y_train = train_df[target_col]

X_test = test_df[feature_cols]
y_test = test_df[target_col]


2. 准备训练、验证和测试数据
  选择的特征数量: 29
  前5个特征: ['Year_Num', 'MonthNum', 'Month_Sin', 'Month_Cos', 'Lag1_Cases']
  测试集开始日期: 2020-01-01 00:00:00
  训练集形状: (816, 36)
  测试集形状: (204, 36)


In [5]:
# 3. 实现时间序列交叉验证
print("\n3. 实现时间序列交叉验证")

# 3.1 定义评估指标
def evaluate_model(y_true, y_pred):
    """计算多种评估指标"""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    # 处理零值问题以计算MAPE
    mask = y_true != 0
    mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    
    return {
        'RMSE': rmse,
        'MAE': mae,
        'MAPE': mape,
        'R2': r2
    }

# 3.2 设置时间序列交叉验证
tscv = TimeSeriesSplit(n_splits=5)

# 3.3 可视化时间序列交叉验证分割
plt.figure(figsize=(12, 6))
sample_train = train_df[train_df['Region'] == train_df['Region'].unique()[0]].copy()
sample_train = sample_train.sort_values('Date')
train_indices = np.arange(len(sample_train))

# 配色方案
colors = plt.cm.tab10(np.linspace(0, 1, 5))

for i, (train_idx, val_idx) in enumerate(tscv.split(train_indices)):
    # 绘制训练/验证分割线
    plt.scatter(train_idx, [i + 0.5] * len(train_idx), c=colors[i], marker='_', lw=6, label=f'训练集 {i+1}')
    plt.scatter(val_idx, [i + 0.5] * len(val_idx), c=colors[i], marker='_', lw=6, alpha=0.3, label=f'验证集 {i+1}')

# 设置图表属性
if chinese_font is not None:
    plt.title('时间序列交叉验证 - 5折', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('数据索引', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('CV迭代', fontproperties=chinese_font, fontsize=12)
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys(), prop=chinese_font)
else:
    plt.title('时间序列交叉验证 - 5折', fontsize=16)
    plt.xlabel('数据索引', fontsize=12)
    plt.ylabel('CV迭代', fontsize=12)
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())

plt.tight_layout()
plt.savefig(f'{output_dir}/time_series_cv.png', dpi=300)
print(f"  保存图表：{output_dir}/time_series_cv.png")
plt.close()


3. 实现时间序列交叉验证
  保存图表：dengue_xgboost_results/time_series_cv.png


In [6]:
# 4. 训练初始XGBoost模型
print("\n4. 训练初始XGBoost模型")

# 4.1 设置初始参数
print("  设置初始参数...")
initial_params = {
    'n_estimators': 100,
    'max_depth': 5,
    'learning_rate': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'objective': 'reg:squarederror',
    'random_state': 42
}

# 4.2 训练初始模型
print("  训练初始模型...")
initial_model = xgb.XGBRegressor(**initial_params)
initial_model.fit(X_train, y_train)

# 4.3 初始模型交叉验证
print("  进行交叉验证...")
cv_scores = {
    'RMSE': [],
    'MAE': [],
    'MAPE': [],
    'R2': []
}

for train_idx, val_idx in tscv.split(X_train):
    # 分割训练集和验证集
    X_cv_train, X_cv_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_cv_train, y_cv_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    # 训练模型
    cv_model = xgb.XGBRegressor(**initial_params)
    cv_model.fit(X_cv_train, y_cv_train)
    
    # 预测并评估
    y_cv_pred = cv_model.predict(X_cv_val)
    metrics = evaluate_model(y_cv_val, y_cv_pred)
    
    # 保存评估指标
    for metric_name, metric_value in metrics.items():
        cv_scores[metric_name].append(metric_value)

# 计算平均评估指标
cv_mean_scores = {metric: np.mean(scores) for metric, scores in cv_scores.items()}

print("  交叉验证平均评估指标:")
for metric, value in cv_mean_scores.items():
    print(f"    {metric}: {value:.4f}")

# 4.4 在测试集上评估初始模型
print("  在测试集上评估初始模型...")
y_test_pred = initial_model.predict(X_test)
test_metrics = evaluate_model(y_test, y_test_pred)

print("  测试集评估指标:")
for metric, value in test_metrics.items():
    print(f"    {metric}: {value:.4f}")

# 4.5 保存初始模型
joblib.dump(initial_model, f'{output_dir}/initial_xgboost_model.pkl')
print(f"  初始模型已保存至: {output_dir}/initial_xgboost_model.pkl")

# 4.6 特征重要性分析
print("  分析特征重要性...")
feature_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': initial_model.feature_importances_
})
feature_importance = feature_importance.sort_values('Importance', ascending=False)

plt.figure(figsize=(12, 8))
ax = sns.barplot(x='Importance', y='Feature', data=feature_importance.head(15))

# 设置图表属性
if chinese_font is not None:
    plt.title('XGBoost模型特征重要性（前15个）', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('重要性分数', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('特征', fontproperties=chinese_font, fontsize=12)
else:
    plt.title('XGBoost模型特征重要性（前15个）', fontsize=16)
    plt.xlabel('重要性分数', fontsize=12)
    plt.ylabel('特征', fontsize=12)

plt.tight_layout()
plt.savefig(f'{output_dir}/feature_importance.png', dpi=300)
print(f"  保存图表：{output_dir}/feature_importance.png")
plt.close()

# 保存初始模型的测试集预测结果
test_results = test_df[['Region', 'Date', 'Dengue_Cases']].copy()
test_results['Predicted_Cases'] = y_test_pred
test_results['Absolute_Error'] = abs(test_results['Dengue_Cases'] - test_results['Predicted_Cases'])
test_results['Percentage_Error'] = (test_results['Absolute_Error'] / test_results['Dengue_Cases']) * 100
test_results.to_csv(f'{output_dir}/initial_model_predictions.csv', index=False)
print(f"  测试集预测结果已保存至: {output_dir}/initial_model_predictions.csv")


4. 训练初始XGBoost模型
  设置初始参数...
  训练初始模型...
  进行交叉验证...
  交叉验证平均评估指标:
    RMSE: 764.7667
    MAE: 243.4629
    MAPE: 13.1228
    R2: 0.8256
  在测试集上评估初始模型...
  测试集评估指标:
    RMSE: 310.5853
    MAE: 116.0491
    MAPE: 82.8796
    R2: 0.7536
  初始模型已保存至: dengue_xgboost_results/initial_xgboost_model.pkl
  分析特征重要性...
  保存图表：dengue_xgboost_results/feature_importance.png
  测试集预测结果已保存至: dengue_xgboost_results/initial_model_predictions.csv


In [7]:
# 1. 优化模型参数
print("1. 优化模型参数")

# 1.1 定义参数网格
print("  定义参数搜索空间...")
param_grid = {
    'n_estimators': randint(50, 300),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.5, 0.5),
    'colsample_bytree': uniform(0.5, 0.5),
    'gamma': uniform(0, 1)
}

# 1.2 设置自定义评分函数
def rmse_scorer(estimator, X, y):
    """自定义RMSE评分函数（越小越好）"""
    y_pred = estimator.predict(X)
    return -np.sqrt(mean_squared_error(y, y_pred))

# 1.3 使用随机搜索进行参数优化
print("  执行随机搜索（这可能需要一些时间）...")
random_search = RandomizedSearchCV(
    estimator=xgb.XGBRegressor(objective='reg:squarederror', random_state=42),
    param_distributions=param_grid,
    n_iter=20,  # 搜索20组参数组合
    scoring=rmse_scorer,
    cv=tscv,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

# 获取最佳参数
best_params = random_search.best_params_
print("\n  最佳参数:")
for param, value in best_params.items():
    print(f"    {param}: {value}")

# 1.4 使用最佳参数训练模型
print("\n  使用最佳参数训练最终模型...")
best_model = xgb.XGBRegressor(**best_params, objective='reg:squarederror', random_state=42)
best_model.fit(X_train, y_train)

# 保存最佳模型
joblib.dump(best_model, f'{output_dir}/optimized_xgboost_model.pkl')
print(f"  最佳模型已保存至: {output_dir}/optimized_xgboost_model.pkl")

1. 优化模型参数
  定义参数搜索空间...
  执行随机搜索（这可能需要一些时间）...
Fitting 5 folds for each of 20 candidates, totalling 100 fits

  最佳参数:
    colsample_bytree: 0.8049983288913105
    gamma: 0.8331949117361643
    learning_rate: 0.062009396052331626
    max_depth: 3
    n_estimators: 213
    subsample: 0.5911180438940311

  使用最佳参数训练最终模型...
  最佳模型已保存至: dengue_xgboost_results/optimized_xgboost_model.pkl


In [8]:
# 2. 评估优化后的模型性能
print("\n2. 评估优化后的模型性能")

# 2.1 在测试集上评估最佳模型
print("  在测试集上评估最佳模型...")
y_best_pred = best_model.predict(X_test)
best_metrics = evaluate_model(y_test, y_best_pred)

print("  测试集评估指标:")
for metric, value in best_metrics.items():
    print(f"    {metric}: {value:.4f}")

# 2.2 比较初始模型和优化模型
print("\n  比较初始模型和优化模型:")
comparison_df = pd.DataFrame({
    '初始模型': pd.Series(test_metrics),
    '优化模型': pd.Series(best_metrics)
})

print(comparison_df)

# 保存比较结果
comparison_df.to_csv(f'{output_dir}/model_comparison.csv')
print(f"  模型比较结果已保存至: {output_dir}/model_comparison.csv")

# 2.3 可视化比较结果
metrics_to_plot = ['RMSE', 'MAE', 'MAPE']
plt.figure(figsize=(10, 6))

bar_width = 0.35
index = np.arange(len(metrics_to_plot))

bars1 = plt.bar(index, [test_metrics[m] for m in metrics_to_plot], bar_width, 
                label='初始模型', color='steelblue')
bars2 = plt.bar(index + bar_width, [best_metrics[m] for m in metrics_to_plot], bar_width,
                label='优化模型', color='lightcoral')

# 设置图表属性
if chinese_font is not None:
    plt.title('初始模型与优化模型性能比较', fontproperties=chinese_font, fontsize=16)
    plt.ylabel('评估指标值', fontproperties=chinese_font, fontsize=12)
    plt.xticks(index + bar_width / 2, metrics_to_plot, fontproperties=chinese_font)
    plt.legend(prop=chinese_font)
else:
    plt.title('初始模型与优化模型性能比较', fontsize=16)
    plt.ylabel('评估指标值', fontsize=12)
    plt.xticks(index + bar_width / 2, metrics_to_plot)
    plt.legend()

# 在柱状图上添加数值标签
def add_labels(bars):
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2f}', ha='center', va='bottom')

add_labels(bars1)
add_labels(bars2)

plt.tight_layout()
plt.savefig(f'{output_dir}/model_performance_comparison.png', dpi=300)
print(f"  保存图表：{output_dir}/model_performance_comparison.png")
plt.close()



2. 评估优化后的模型性能
  在测试集上评估最佳模型...
  测试集评估指标:
    RMSE: 418.6332
    MAE: 159.7621
    MAPE: 118.2177
    R2: 0.5524

  比较初始模型和优化模型:
            初始模型        优化模型
RMSE  310.585297  418.633249
MAE   116.049088  159.762115
MAPE   82.879647  118.217731
R2      0.753646    0.552425
  模型比较结果已保存至: dengue_xgboost_results/model_comparison.csv
  保存图表：dengue_xgboost_results/model_performance_comparison.png


In [9]:
# 3. 可视化预测结果与实际值比较
print("\n3. 可视化预测结果与实际值比较")

# 3.1 准备结果数据
test_results = test_df[['Region', 'Date', 'Dengue_Cases']].copy()
test_results['Predicted_Cases'] = y_best_pred
test_results['Absolute_Error'] = abs(test_results['Dengue_Cases'] - test_results['Predicted_Cases'])
test_results.to_csv(f'{output_dir}/optimized_model_predictions.csv', index=False)

# 3.2 按地区绘制预测结果
# 获取前5个高发地区
top_regions = df.groupby('Region')['Dengue_Cases'].sum().sort_values(ascending=False).head(5).index.tolist()

for region in top_regions:
    region_results = test_results[test_results['Region'] == region].sort_values('Date')
    
    plt.figure(figsize=(14, 7))
    
    # 绘制实际值和预测值
    plt.plot(region_results['Date'], region_results['Dengue_Cases'], 'b-', marker='o', label='实际值')
    plt.plot(region_results['Date'], region_results['Predicted_Cases'], 'r--', marker='x', label='预测值')
    
    # 绘制误差区域
    plt.fill_between(region_results['Date'], 
                    region_results['Dengue_Cases'], 
                    region_results['Predicted_Cases'], 
                    alpha=0.3, color='gray', label='误差')
    
    # 设置图表属性
    if chinese_font is not None:
        plt.title(f'{region} - XGBoost预测结果与实际值比较', fontproperties=chinese_font, fontsize=16)
        plt.xlabel('日期', fontproperties=chinese_font, fontsize=12)
        plt.ylabel('登革热病例数', fontproperties=chinese_font, fontsize=12)
        plt.legend(prop=chinese_font)
    else:
        plt.title(f'{region} - XGBoost预测结果与实际值比较', fontsize=16)
        plt.xlabel('日期', fontsize=12)
        plt.ylabel('登革热病例数', fontsize=12)
        plt.legend()
    
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    # 保存图表
    plt.savefig(f'{output_dir}/{region}_prediction_comparison.png', dpi=300)
    print(f"  保存图表：{output_dir}/{region}_prediction_comparison.png")
    plt.close()

# 3.3 全国数据预测结果
national_test = test_results.groupby('Date').agg({
    'Dengue_Cases': 'sum',
    'Predicted_Cases': 'sum'
}).reset_index()

plt.figure(figsize=(14, 7))

# 绘制实际值和预测值
plt.plot(national_test['Date'], national_test['Dengue_Cases'], 'b-', marker='o', label='实际值')
plt.plot(national_test['Date'], national_test['Predicted_Cases'], 'r--', marker='x', label='预测值')

# 绘制误差区域
plt.fill_between(national_test['Date'], 
                national_test['Dengue_Cases'], 
                national_test['Predicted_Cases'], 
                alpha=0.3, color='gray', label='误差')

# 设置图表属性
if chinese_font is not None:
    plt.title('全国 - XGBoost预测结果与实际值比较', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('日期', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('登革热病例数', fontproperties=chinese_font, fontsize=12)
    plt.legend(prop=chinese_font)
else:
    plt.title('全国 - XGBoost预测结果与实际值比较', fontsize=16)
    plt.xlabel('日期', fontsize=12)
    plt.ylabel('登革热病例数', fontsize=12)
    plt.legend()

plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

# 保存图表
plt.savefig(f'{output_dir}/national_prediction_comparison.png', dpi=300)
print(f"  保存图表：{output_dir}/national_prediction_comparison.png")
plt.close()

# 3.4 预测值与实际值散点图
plt.figure(figsize=(10, 10))

# 绘制散点图
plt.scatter(test_results['Dengue_Cases'], test_results['Predicted_Cases'], alpha=0.5)

# 添加理想预测线
max_value = max(test_results['Dengue_Cases'].max(), test_results['Predicted_Cases'].max())
plt.plot([0, max_value], [0, max_value], 'r--', label='理想预测线')

# 设置图表属性
if chinese_font is not None:
    plt.title('实际值与预测值散点图', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('实际登革热病例数', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('预测登革热病例数', fontproperties=chinese_font, fontsize=12)
    plt.legend(prop=chinese_font)
else:
    plt.title('实际值与预测值散点图', fontsize=16)
    plt.xlabel('实际登革热病例数', fontsize=12)
    plt.ylabel('预测登革热病例数', fontsize=12)
    plt.legend()

plt.grid(True)
plt.axis('equal')
plt.tight_layout()

# 保存图表
plt.savefig(f'{output_dir}/actual_vs_predicted_scatter.png', dpi=300)
print(f"  保存图表：{output_dir}/actual_vs_predicted_scatter.png")
plt.close()

# 3.5 预测误差分布
plt.figure(figsize=(12, 6))

# 计算预测误差
errors = test_results['Dengue_Cases'] - test_results['Predicted_Cases']

# 绘制误差直方图
sns.histplot(errors, kde=True)

# 添加垂直线标记零误差
plt.axvline(x=0, color='r', linestyle='--')

# 设置图表属性
if chinese_font is not None:
    plt.title('预测误差分布', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('预测误差（实际值-预测值）', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('频率', fontproperties=chinese_font, fontsize=12)
else:
    plt.title('预测误差分布', fontsize=16)
    plt.xlabel('预测误差（实际值-预测值）', fontsize=12)
    plt.ylabel('频率', fontsize=12)

plt.grid(True)
plt.tight_layout()

# 保存图表
plt.savefig(f'{output_dir}/prediction_error_distribution.png', dpi=300)
print(f"  保存图表：{output_dir}/prediction_error_distribution.png")
plt.close()



3. 可视化预测结果与实际值比较
  保存图表：dengue_xgboost_results/Region IV-A_prediction_comparison.png
  保存图表：dengue_xgboost_results/Region III_prediction_comparison.png
  保存图表：dengue_xgboost_results/Region VI_prediction_comparison.png
  保存图表：dengue_xgboost_results/NCR_prediction_comparison.png
  保存图表：dengue_xgboost_results/Region VII_prediction_comparison.png
  保存图表：dengue_xgboost_results/national_prediction_comparison.png
  保存图表：dengue_xgboost_results/actual_vs_predicted_scatter.png
  保存图表：dengue_xgboost_results/prediction_error_distribution.png


In [10]:
# 4. 比较XGBoost与前面的模型（如果有）
print("\n4. 比较XGBoost与前面的模型")

try:
    # 尝试加载之前的模型评估结果
    previous_models = pd.read_csv('dengue_forecast_results/model_comparison.csv', index_col=0)
    
    # 添加XGBoost模型结果
    previous_models['XGBoost'] = pd.Series(best_metrics)
    
    # 保存更新后的比较结果
    previous_models.to_csv(f'{output_dir}/all_models_comparison.csv')
    print(f"  所有模型比较结果已保存至: {output_dir}/all_models_comparison.csv")
    
    # 可视化比较结果
    metrics_to_plot = ['RMSE', 'MAE', 'MAPE']
    
    for metric in metrics_to_plot:
        plt.figure(figsize=(10, 6))
        
        # 提取该指标的所有模型结果
        model_values = previous_models.loc[metric].sort_values()
        
        # 绘制条形图
        bars = plt.bar(range(len(model_values)), model_values.values, color='skyblue')
        
        # 添加数值标签
        for i, bar in enumerate(bars):
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + 0.01*max(model_values),
                    f'{height:.2f}', ha='center', va='bottom')
        
        # 设置图表属性
        if chinese_font is not None:
            plt.title(f'不同模型的{metric}比较', fontproperties=chinese_font, fontsize=16)
            plt.ylabel(metric, fontproperties=chinese_font, fontsize=12)
            plt.xticks(range(len(model_values)), model_values.index, fontproperties=chinese_font, rotation=45)
        else:
            plt.title(f'不同模型的{metric}比较', fontsize=16)
            plt.ylabel(metric, fontsize=12)
            plt.xticks(range(len(model_values)), model_values.index, rotation=45)
        
        plt.grid(axis='y')
        plt.tight_layout()
        
        # 保存图表
        plt.savefig(f'{output_dir}/all_models_{metric}_comparison.png', dpi=300)
        print(f"  保存图表：{output_dir}/all_models_{metric}_comparison.png")
        plt.close()
    
except Exception as e:
    print(f"  无法加载之前的模型比较结果: {e}")
    print("  仅保存XGBoost模型结果")


4. 比较XGBoost与前面的模型
  所有模型比较结果已保存至: dengue_xgboost_results/all_models_comparison.csv
  无法加载之前的模型比较结果: 'RMSE'
  仅保存XGBoost模型结果


<Figure size 1000x600 with 0 Axes>